<a href="https://colab.research.google.com/github/aminramezani345/-Clinical-Trials-for-Evidence-Based-Medicine-in-Spanish-CT-EBM-SP-Corpus/blob/main/ROCT_Net_OCTID_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [3]:
!pip install git+https://github.com/keras-team/keras-contrib

  Cloning https://github.com/keras-team/keras-contrib to /tmp/pip-req-build-jxar082r
  Running command git clone --filter=blob:none --quiet https://github.com/keras-team/keras-contrib /tmp/pip-req-build-jxar082r
  Resolved https://github.com/keras-team/keras-contrib to commit 3fc5ef709e061416f4bc8a92ca3750c824b5d2b0
  Preparing metadata (setup.py) ... done
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-py3-none-any.whl size=101055 sha256=88c3ffc75fa20b73cc5221032dda8575f661b11b797d5cc2c3fa6c7fe0804f93
  Stored in directory: /tmp/pip-ephem-wheel-cache-is0ud_7q/wheels/81/0e/79/8ddbf0372cbfb49da9695e8774d7b4685260fe43bb597a10da
Successfully built keras-contrib


In [4]:
pip install tensorflow_addons

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 5.5 MB/s eta 0:00:00


In [9]:
import numpy as np
from tensorflow.keras.preprocessing import image
import tensorflow as tf
from keras_contrib.layers import Capsule
from keras_contrib.activations import squash
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import cv2
import zipfile
import shutil
import random
import pandas as pd
import csv
import os
import tensorflow as tf
from keras_contrib.layers import Capsule
from keras_contrib.activations import squash

In [10]:
!gdown --id "1okQhlbA6RNJ23fQG_7z-zcZEiCfCT8pb" #if this code failed, get the dataset from https://drive.google.com/file/d/1okQhlbA6RNJ23fQG_7z-zcZEiCfCT8pb/view?usp=sharing

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1okQhlbA6RNJ23fQG_7z-zcZEiCfCT8pb
From (redirected): https://drive.google.com/uc?id=1okQhlbA6RNJ23fQG_7z-zcZEiCfCT8pb&confirm=t&uuid=23f584dc-e946-4368-965a-83487ac81b17
To: /content/OCTA-gholami.zip
100% 45.7M/45.7M [00:01<00:00, 36.9MB/s]


In [18]:
archive = zipfile.ZipFile('OCTA-gholami.zip')
for file in archive.namelist():
     archive.extract(file, '.')

FileNotFoundError: [Errno 2] No such file or directory: 'OCTA-gholami.zip'

In [12]:
train_datagen = keras.preprocessing.image.ImageDataGenerator(horizontal_flip=True,vertical_flip=True
                                                             ,zoom_range=0.1,rotation_range=360
                                                             ,width_shift_range=0.1,height_shift_range=0.1)

test_datagen = keras.preprocessing.image.ImageDataGenerator()

train_df = pd.read_csv("train_gholami.csv")
test_df = pd.read_csv("test_gholami.csv")

In [19]:
#create dataloader
shape=(512, 512)
batch_size=8 #increase when having better GPU
train_generator = train_datagen.flow_from_dataframe(
      dataframe=train_df,
      directory='gholami',
      x_col="filename",
      y_col="class",
      target_size=shape,
      batch_size=batch_size,
      class_mode='categorical',shuffle=True)
validation_generator = test_datagen.flow_from_dataframe(
        dataframe=test_df,
        directory='gholami',
        x_col="filename",
        y_col="class",
        target_size=shape,
        batch_size=batch_size,
        class_mode='categorical',shuffle=True)
train_img_num=len(train_generator.filenames)

Found 0 validated image filenames belonging to 0 classes.
Found 0 validated image filenames belonging to 0 classes.


/usr/local/lib/python3.10/dist-packages/keras/src/preprocessing/image.py:1137: UserWarning: Found 459 invalid image filename(s) in x_col="filename". These filename(s) will be ignored.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/preprocessing/image.py:1137: UserWarning: Found 113 invalid image filename(s) in x_col="filename". These filename(s) will be ignored.
  warnings.warn(


In [20]:
!git clone https://github.com/mhrahimzadeh1374/automl

Cloning into 'automl'...
remote: Enumerating objects: 3922, done.
remote: Total 3922 (delta 0), reused 0 (delta 0), pack-reused 3922
Receiving objects: 100% (3922/3922), 23.16 MiB | 17.35 MiB/s, done.
Resolving deltas: 100% (2960/2960), done.


In [21]:
cd automl/efficientnetv2

/content/automl/efficientnetv2/automl/efficientnetv2


In [25]:
from effnetv2_model import get_model

In [26]:
cd ../../

/content


In [28]:
name="ROCT-Net_OCTID dataset"
!mkdir "models"
keras.backend.clear_session()
input_tensor=keras.Input(shape=(shape[0],shape[1],3))
base_model1=get_model('efficientnetv2-b0', include_top=False, pretrained=True)(input_tensor) #load EfficientNetV2-B0
base_model2=keras.applications.Xception(input_tensor=input_tensor,weights='imagenet',include_top=False)(input_tensor) #load Xception

concatenated=keras.layers.concatenate([base_model1,base_model2]) #load concatenated model

avg=keras.layers.AveragePooling2D(3,padding='valid')(concatenated) #deploy Wise-srNet
depthw=keras.layers.DepthwiseConv2D(5,
                                      depthwise_initializer=keras.initializers.RandomNormal(mean=0.0,stddev=0.01),
                                      bias_initializer=keras.initializers.Zeros(),depthwise_constraint=keras.constraints.NonNeg())(avg)
# Define Capsules
capsule = Capsule(num_capsule=10,
                dim_capsule=16,
                routings=3,
                activation=squash,
                share_weights=True)(depthw)

flat=keras.layers.Flatten()(capsule)
dp=keras.layers.Dropout(0.2)(flat)
preds=keras.layers.Dense(5,activation='softmax',
                          kernel_initializer=keras.initializers.RandomNormal(mean=0.0,stddev=0.01),
                          bias_initializer=keras.initializers.Zeros(),)(dp)
model=keras.Model(inputs=input_tensor, outputs=preds)

##################################
for layer in model.layers:
  layer.trainable = True
model.summary()
filepath="models/%s-{epoch:02d}-{val_accuracy:.4f}.hdf5"%name

#Note that keras classic saving function would not save capsule weights, so the saved model will not be as equal as the trained model. This bug must be fixed in the future.

checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', save_best_only=False, mode='max',save_weights_only=True) #creating checkpoint to save the best validation accuracy
callbacks_list = [checkpoint]

lr_schedule =keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.045,
    decay_steps=2*int(train_img_num/batch_size),
    decay_rate=0.94,
    staircase=True)
optimizer=keras.optimizers.SGD(momentum=0.9,learning_rate=lr_schedule)
model.compile(optimizer=optimizer, loss='categorical_crossentropy',metrics=['accuracy'])

hist=model.fit_generator(train_generator, epochs=40,validation_data=validation_generator,shuffle=True,callbacks=callbacks_list) #start training
with open('{}-results.csv'.format(name), mode='w',newline='') as csv_file: #write evaluation metrics
  csv_writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
  for key in hist.history:
    data=[key]
    data.extend(hist.history[key])
    csv_writer.writerow(data)

mkdir: cannot create directory ‘models’: File exists
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 512, 512, 3)]        0         []                            
                                                                                                  
 efficientnetv2-b0 (EffNetV  multiple                     5919312   ['input_1[0][0]']             
 2Model)                                                                                          
                                                                                                  
 xception (Functional)       (None, 16, 16, 2048)         2086148   ['input_1[0][0]']             
                                                          0                                       
                                         

<ipython-input-28-058944e64a83>:47: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  hist=model.fit_generator(train_generator, epochs=40,validation_data=validation_generator,shuffle=True,callbacks=callbacks_list) #start training


ValueError: Asked to retrieve element 0, but the Sequence has length 0

In [29]:
checkpoint_file = '/content/models/ROCT-Net_OCTID dataset-06-0.6903.hdf5'
model.load_weights(checkpoint_file)

FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = '/content/models/ROCT-Net_OCTID dataset-06-0.6903.hdf5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [23]:
def preprocess_image(img_path, target_size):
    img = image.load_img(img_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0  # Normalize the image
    return img_array

# Path to the image in your test folder
image_path = '/content/gholami/AMRD1.jpeg'

# Preprocess the image
img = preprocess_image(image_path, target_size=(512, 512))


In [1]:
# Define path to the folder containing test images in Google Drive
test_folder = '/content/testOCT/'

# List all files in the test folder
test_files = os.listdir(test_folder)
for filename in testOCT:
    if filename.endswith(".jpg") or filename.endswith(".png"):  # Adjust file extensions as needed
        # Construct full path to the image
        img_path = os.path.join(test_folder, filename)

        # Preprocess the image
        img = preprocess_image(img_path, target_size=(512, 512))

        # Make predictions
        predictions = model.predict(img)
        predicted_class = np.argmax(predictions, axis=1)

        # Print or store the predicted class
        print(f'Image: {filename}, Predicted class: {class_labels[predicted_class[0]]}')

NameError: name 'os' is not defined